<a href="https://colab.research.google.com/github/SiddamVamsi264/Text-Emotion-Classifier/blob/main/Text_Emotions_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import files

uploaded = files.upload()

Saving train.txt to train.txt


After running the cell above and uploading your `train.txt` file, you can then run the code cell with your data loading code again.

In [12]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense


data = pd.read_csv("train.txt", sep=';')
data.columns = ["Text", "Emotions"]
print(data.head())


                                                Text Emotions
0  i can go from feeling so hopeless to so damned...  sadness
1   im grabbing a minute to post i feel greedy wrong    anger
2  i am ever feeling nostalgic about the fireplac...     love
3                               i am feeling grouchy    anger
4  ive been feeling a little burdened lately wasn...  sadness


As this is a problem of natural language processing, I’ll start by tokenizing the data

In [13]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

Now,  pad the sequences to the same length to feed them into a neural network. Here’s how we can pad the sequences of the texts to have the same length

In [14]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

Now, label encoder method is used to convert the classes from strings to a numerical representation

In [15]:
# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

One-hot encode the labels. One hot encoding refers to the transformation of categorical labels into a binary representation where each label is represented as a vector of all zeros except a single 1. This is necessary because machine learning algorithms work with numerical data. So here is how we can One-hot encode the labels

In [16]:
# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

**Text Emotions Classification Model**

Now we will split the data into training and test sets:

In [18]:
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences,
                                                one_hot_labels,
                                                test_size=0.2)

let’s define a neural network architecture for our classification problem and use it to train a model to classify emotions:

In [19]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1,
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))

Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


400/400 ━━━━━━━━━━━━━━━━━━━━ 16s 36ms/step - accuracy: 0.3731 - loss: 1.5447 - val_accuracy: 0.7197 - val_loss: 0.8509
Epoch 2/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 20s 34ms/step - accuracy: 0.8467 - loss: 0.4960 - val_accuracy: 0.8147 - val_loss: 0.5654
Epoch 3/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 21s 35ms/step - accuracy: 0.9845 - loss: 0.0668 - val_accuracy: 0.8244 - val_loss: 0.6081
Epoch 4/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 14s 34ms/step - accuracy: 0.9950 - loss: 0.0290 - val_accuracy: 0.8222 - val_loss: 0.6696
Epoch 5/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 14s 35ms/step - accuracy: 0.9979 - loss: 0.0122 - val_accuracy: 0.8231 - val_loss: 0.6677
Epoch 6/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 20s 34ms/step - accuracy: 0.9972 - loss: 0.0119 - val_accuracy: 0.8194 - val_loss: 0.7102
Epoch 7/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 20s 34ms/step - accuracy: 0.9981 - loss: 0.0096 - val_accuracy: 0.8194 - val_loss: 0.7381
Epoch 8/10
400/400 ━━━━━━━━━━━━━━━━━━━━ 13s 33ms/step - accuracy: 0.9981 - loss: 0.0079 - val_accurac

Now let’s take a sentence as an input text and see how the model performs

In [20]:
input_text = "She didn't come today because she lost her dog yestertay!"

# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
['sadness']
